In [1]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import op
import script


### Exercise 1

Write the `op_hash160` function.

#### Make [this test](/edit/code-ch06/op.py) pass: `op.py:OpTest:test_op_hash160`

In [ ]:
# Exercise 1

reload(op)
run(op.OpTest("test_op_hash160"))

In [2]:
from script import Script
z = 0x7c076ff316692a3d7eb3c3bb0f8b1488cf72e1afcd929e29307032997a838a3d
sec = bytes.fromhex('04887387e452b8eacc4acfde10d9aaf7f6d9a0f975aabb10d006e4da568744d06c61de6d95231cd89026e286df3b6ae4a894a3378e393e93a0f45b666329a0ae34')
sig = bytes.fromhex('3045022000eff69ef2b1bd93a66ed5219add4fb51e11a840f404876325a1e8ffe0529a2c022100c7207fee197d27c618aea621406f6bf5ef6fca38681d82b2f06fddbdce6feab601')
script_pubkey = Script([sec, 0xac])
script_sig = Script([sig])
combined_script = script_sig + script_pubkey
print('\n'.join(combined_script.__repr__().split(' ')))
print(combined_script.evaluate(z))


3045022000eff69ef2b1bd93a66ed5219add4fb51e11a840f404876325a1e8ffe0529a2c022100c7207fee197d27c618aea621406f6bf5ef6fca38681d82b2f06fddbdce6feab601
04887387e452b8eacc4acfde10d9aaf7f6d9a0f975aabb10d006e4da568744d06c61de6d95231cd89026e286df3b6ae4a894a3378e393e93a0f45b666329a0ae34
OP_CHECKSIG
True


### Exercise 2

Write the `op_checksig` function in `op.py`

#### Make [this test](/edit/code-ch06/op.py) pass: `op.py:OpTest:test_op_checksig`

In [ ]:
# Exercise 2

reload(op)
run(op.OpTest("test_op_checksig"))

### Exercise 3

Create a ScriptSig that can unlock this ScriptPubKey. Note `OP_MUL` multiplies the top two elements of the stack.

`767695935687`

* `56 = OP_6`
* `76 = OP_DUP`
* `87 = OP_EQUAL`
* `93 = OP_ADD`
* `95 = OP_MUL`

In [9]:
"""Note: Implementing stack as a list can cause variable performance for append operations.
There is another iterable, deque, that is optimized for access to head and tail of the list.
The append operation could be renamed to a more contextual `push`:

from collections import deque
class Stack(deque):
    def push(self, obj):
        self.append(obj)

"""
# Exercise 3
#             op       stack[btm,top]  desc
# begin ScriptSig
#             OP_X     [x]              # push x
# begin ScriptPubKey
#             OP_DUP   [x, x]           # push copy of top of stack
#             OP_DUP   [x, x, x]        # push copy of top of stack 
#             OP_MUL   [x, x**2]        # push product of top 2 of stack
#             OP_ADD   [x + x**2]       # push sum of top 2 of stack
#             OP_6     [x + x**2, 6]    # push 6
#             OP_EQUAL [?]              # pop twice, push 1 if equal, 0 if not
# -3 or 2 could solve for x
from op import encode_num
from script import Script
script_pubkey = Script([0x76, 0x76, 0x95, 0x93, 0x56, 0x87])
# [(solve with 2), (solve with -3), (solve with 3 * -1), (solve with (76 byte num+3) - 76 byte num)]
# 76 byte nums that would require op_pushdata1:
bignum = 4165788178955036068153632992568381390820338178251230770191718730983557332241978374424803939335131652305014348911737262865463367354138880309094274343267351779951617810473672306131200
bignum_plus_3 = 4165788178955036068153632992568381390820338178251230770191718730983557332241978374424803939335131652305014348911737262865463367354138880309094274343267351779951617810473672306131203
script_sigs = [ Script([0x52]),  # 82: OP_2
                Script([encode_num(-3)]),  # 0x83/131: Script encoded -3 (would be serialized with
                                           #   0x01 prefix) 
                # Script([0x53,    # 83: OP_3
                #         0x4f,    # 79: OP_1NEGATE
                #         0x95]),  # 149: OP_MUL
                # OP_MUL is dsiabled in Bitcoin Core and Song's code after ch06
                Script([0x53,    # 83: OP_3
                        0x8f]),  # 143: OP_NEGATE
                Script([encode_num(bignum),
                        encode_num(bignum_plus_3),         
                        0x94]) ]                     # 148: OP_SUB
for script_sig in script_sigs:
    combined_script = script_sig + script_pubkey
    print(combined_script.evaluate(0))


True
True
True
True


### Exercise 4

Figure out what this Script is doing:

`6e879169a77ca787`

* `69 = OP_VERIFY`
* `6e = OP_2DUP`
* `7c = OP_SWAP`
* `87 = OP_EQUAL`
* `91 = OP_NOT`
* `a7 = OP_SHA1`

Use the `Script.parse` method and look up what various opcodes do at https://en.bitcoin.it/wiki/Script

In [22]:
# Exercise 4
#             op         stack[btm,top]       desc
# begin ScriptSig
#                        [x, y]          
# begin ScriptPubKey
#             OP_2DUP    [x, y, x, y]        # push copy of top 2 stack items
#             OP_EQUAL   [1|0, x, y]         # push 1 if top 2 elements equal, 0 if not 
#             OP_NOT     [0|1, x, y]         # push opposite if 0 or 1 popped
#             OP_VERIFY  [x, y]              # invalidate transaction if 0 at top of stack
#             OP_SHA1    [sha1(x), y]        # push sha1 of top of stack
#             OP_SWAP    [y, sha1(x)]        # two items on the stack are swapped
#             OP_SHA1    [sha1(y), sha1(x)]  # push sha1 of top of stack
#             OP_EQUAL   [1|0]               # push 1 if top 2 elements equal, 0 if not

# Need two different inputs that hash to the same value, or a sha1 collision.
# Famous Google-discovered example at https://shattered.it/
# $ curl -sSO https://shattered.it/shattered-1.pdf
# $ curl -sSO https://shattered.it/shattered-2.pdf

# Song also references the same research, but generates much shorter collision examples from:
# https://security.googleblog.com/2017/02/announcing-first-sha1-collision.html ->
#    https://marc-stevens.nl/research/papers/SBKAM17-SHAttered.pdf
# modified from answers.py:
prefix_hex = '255044462d312e330a25e2e3cfd30a0a0a312030206f626a0a3c3c2f576964746820\
32203020522f4865696768742033203020522f547970652034203020522f537562747970652035\
203020522f46696c7465722036203020522f436f6c6f7253706163652037203020522f4c656e67\
74682038203020522f42697473506572436f6d706f6e656e7420383e3e0a73747265616d0affd8\
fffe00245348412d3120697320646561642121212121852fec092339759c39b1a1c63c4c97e1ff\
fe01'
M_1_1_hex = '7f46dc93a6b67e013b029aaa1db2560b\
45ca67d688c7f84b8c4c791fe02b3df6\
14f86db1690901c56b45c1530afedfb7\
6038e972722fe7ad728f0e4904e046c2'
M_2_1_hex = '30570fe9d41398abe12ef5bc942be335\
42a4802d98b5d70f2a332ec37fac3514\
e74ddc0f2cc1a874cd0c78305a215664\
61309789606bd0bf3f98cda8044629a1'
M_1_2_hex = '7346dc9166b67e118f029ab621b2560f\
f9ca67cca8c7f85ba84c79030c2b3de2\
18f86db3a90901d5df45c14f26fedfb3\
dc38e96ac22fe7bd728f0e45bce046d2'
M_2_2_hex = '3c570feb141398bb552ef5a0a82be331\
fea48037b8b5d71f0e332edf93ac3500\
eb4ddc0decc1a864790c782c76215660\
dd309791d06bd0af3f98cda4bc4629b1'

from random import randint
from script import Script

script_pubkey = Script([0x6e, 0x87, 0x91, 0x69, 0xa7, 0x7c, 0xa7, 0x87])
prefix = bytes.fromhex(prefix_hex)
M_1_1 = bytes.fromhex(M_1_1_hex)
M_2_1 = bytes.fromhex(M_2_1_hex)
M_1_2 = bytes.fromhex(M_1_2_hex)
M_2_2 = bytes.fromhex(M_2_2_hex)

print('shared prefix + colliding segments +')
for _ in range(10):
    # generate random bytes as suffix
    suffix = bytearray()
    for __ in range(randint(0, 100)):
        suffix.append(randint(0, 255))
    print(f'\t random {len(suffix)} byte suffix: ', end='')
    script_sig = Script([prefix + M_1_1 + M_2_1 + bytes(suffix),
                        prefix + M_1_2 + M_2_2 + bytes(suffix)])
    combined_script = script_sig + script_pubkey
    print(combined_script.evaluate(0))    

print('shattered-1.pdf vs shattered-2.pdf: ', end='')
with open('shattered-1.pdf', 'rb') as file1, open('shattered-2.pdf', 'rb') as file2:
    script_sig = Script([file1.read(), file2.read()])
    combined_script = script_sig + script_pubkey
    print(combined_script.evaluate(0))

shared prefix + colliding segments +
	 random 84 byte suffix: True
	 random 18 byte suffix: True
	 random 44 byte suffix: True
	 random 92 byte suffix: True
	 random 9 byte suffix: True
	 random 57 byte suffix: True
	 random 22 byte suffix: True
	 random 89 byte suffix: True
	 random 20 byte suffix: True
	 random 17 byte suffix: True
shattered-1.pdf vs shattered-2.pdf: True
